<a href="https://colab.research.google.com/github/Tomalison/hello-colab/blob/main/%E5%B0%88%E9%A1%8C%E5%AF%A6%E4%BD%9C_07_%E7%88%AC%E8%9F%B2%E8%B3%87%E6%96%99%E8%92%90%E9%9B%86.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

import sqlite3
import datetime

import requests
import json
# 連線資料庫
conn = sqlite3.connect('example.db')
c = conn.cursor()

# 建立 pharmacies 表格，如果表格不存在的話
c.execute('''CREATE TABLE IF NOT EXISTS pharmacies
             (city text, counts text, createdAt datetime)''')
c.execute('''DELETE FROM pharmacies''')
conn.commit()

# 新增原始資料
URL = 'https://raw.githubusercontent.com/kiang/pharmacies/master/json/points.json'
response = requests.get(URL)
d = response.text
data = json.loads(d)

# 整理資料
med_count = {}
for d in data['features']:
  county = d['properties']["address"][0:3]
  med_count[county] = med_count.setdefault(county,0) + 1

# 加入資料
for city,counts in med_count.items():
  t = datetime.datetime.now()
#  print(f"INSERT INTO stocks VALUES ('{k}', '{v}', '{t}')")
  c.execute(f"INSERT INTO pharmacies VALUES ('{city}', {counts}, '{t}')")
  conn.commit()

# 查詢資料
c.execute("SELECT * FROM pharmacies")
print(c.fetchall())


# 儲存變更
conn.commit()

# 關閉資料庫連線
conn.close()